In [11]:
import pandas as pd
import numpy as np

In [12]:
df_category = pd.read_csv('category.csv')
df_customers = pd.read_csv('customers.csv')
df_geolocation = pd.read_csv('geolocation.csv')
df_order_items = pd.read_csv('order_items.csv')
df_order_payments = pd.read_csv('order_payments.csv')
df_order_reviews = pd.read_csv('order_reviews.csv')
df_products = pd.read_csv('products.csv')
df_sellers = pd.read_csv('sellers.csv')

#inserir o nome da base de teste 
df_teste = pd.read_csv('amostra_novos_pedidos.csv')

In [13]:
df_geolocation_c = df_geolocation.rename(columns={
    'geolocation_zip_code_prefix': 'customer_zip_code',
    'geolocation_lat': 'geolocation_lat_c',
    'geolocation_lng': 'geolocation_lng_c',
    'geolocation_city': 'geolocation_city_c',
    'geolocation_state': 'geolocation_state_c'
})

df_geolocation_c = df_geolocation_c.drop_duplicates()
df = df_teste.merge(df_geolocation_c, on='customer_zip_code', how='left')
df = df.drop_duplicates(subset= ['order_id','order_item_id'])

In [14]:
df_products = df_products.drop_duplicates(subset=['product_id'])
df = df.merge(df_products, on='product_id', how='left')

In [15]:
df_sellers = df_sellers.drop_duplicates(subset=['seller_id'])
df = df.merge(df_sellers, on='seller_id', how='left')

In [16]:
df_geolocation_s = df_geolocation.rename(columns={
    'geolocation_zip_code_prefix': 'seller_zip_code_prefix',
    'geolocation_lat': 'geolocation_lat_s',
    'geolocation_lng': 'geolocation_lng_s',
    'geolocation_city': 'geolocation_city_s',
    'geolocation_state': 'geolocation_state_s'
})
df_geolocation_s = df_geolocation_s.drop_duplicates()
df = df.merge(df_geolocation_s, on='seller_zip_code_prefix', how='left')
df = df.drop_duplicates(subset= ['order_id','order_item_id'])

In [17]:

df['product_volume'] = df['product_length_cm'] * df['product_height_cm'] * df['product_width_cm']

# Função para calcular a distância usando a fórmula de Haversine
def calcular_distancia(lat1, lon1, lat2, lon2):
    # Converter as coordenadas de graus para radianos
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças das coordenadas
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = np.sin(dlat / 2)*2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)*2
    a = (a**2)**(1/2)
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Raio da Terra em quilômetros
    raio_terra_km = 6371
    distancia = raio_terra_km * c
    return distancia

# Aplicar a função ao DataFrame para criar a nova coluna com as distâncias
df['distancia_km'] = df.apply(lambda row: calcular_distancia(row['geolocation_lat_c'], row['geolocation_lng_c'], row['geolocation_lat_s'], row['geolocation_lng_s']), axis=1)
df = df.drop(columns=['geolocation_lat_c', 'geolocation_lng_c', 'geolocation_lat_s', 'geolocation_lng_s'])

# Converter colunas de timestamp para datetime
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], format='mixed', dayfirst=False)
df['order_approved_at'] = pd.to_datetime(df['order_approved_at'],format='mixed',dayfirst=False)

# Calcular o tempo de aprovação da compra em minutos
df['purchase_to_approval_minutes'] = (df['order_approved_at'] - df['order_purchase_timestamp']).dt.total_seconds() / 86400

# Dicionário de substituições para padronizar variações de nomes comuns
substituicoes = {
    "sp": "sao paulo",
    "sao paulo": "sao paulo",
    "so paulo": "sao paulo",
    "Sao paulo": "sao paulo",
    "saopaulo": "sao paulo",
    "são paulo": "sao paulo",
    "saopaulo": "sao paulo",
    "varzea paulista": "varzea paulista"
    # Adicione outras variações de cidade aqui, se necessário
}

# Função para aplicar as substituições específicas
def padronizar_cidade(nome):
    return substituicoes.get(nome, nome)  # Substitui se a cidade estiver no dicionário, caso contrário, mantém o original

# Aplicando a normalização e padronização nas colunas de cidades
df['geolocation_city_c'] = df['geolocation_city_c'].apply(padronizar_cidade)
df['geolocation_city_s'] = df['geolocation_city_s'].apply(padronizar_cidade)

In [18]:
# Remover duplicatas com base em 'order_id' e 'product_id'
df = df.drop_duplicates(subset=['order_id', 'product_id'])

# Ordenar o DataFrame por 'order_id', 'distancia_km', 'purchase_to_approval_minutes', 'freight_value' e 'product_volume' em ordem decrescente
df = df.sort_values(
    by=['order_id', 'distancia_km', 'purchase_to_approval_minutes', 'freight_value', 'product_volume'], 
    ascending=[False, False, False, False, False]
)

# Manter apenas a linha com os maiores valores por 'order_id'
df = df.groupby('order_id').first().reset_index()

In [19]:
df = df.rename(columns={'purchase_to_approval_minutes': 'purchase_to_approval_days'})

In [20]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import OrdinalEncoder

# Passo 1: Preparar o conjunto de treinamento
treino = pd.read_csv('treino_out_deliverytime.csv')

# Tratar valores ausentes nas variáveis numéricas
numeric_cols = treino.select_dtypes(include=['float64', 'int64']).columns
treino[numeric_cols] = treino[numeric_cols].fillna(treino[numeric_cols].mean())

# Tratar valores ausentes nas variáveis categóricas, excluindo 'order_id'
categorical_cols = treino.select_dtypes(include=['object']).columns.drop('order_id', errors='ignore')
treino[categorical_cols] = treino[categorical_cols].fillna(treino[categorical_cols].mode().iloc[0])

# Remover 'order_id' das features
treino = treino.drop(columns=['order_id'], errors='ignore')

# Passo 2: Codificar variáveis categóricas no conjunto de treinamento
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
treino[categorical_cols] = ordinal_encoder.fit_transform(treino[categorical_cols])

# Passo 3: Separar dados para prever 'approval_to_carrier_days' e 'carrier_to_customer_days'
X_intermediate = treino.drop(columns=['approval_to_carrier_days', 'carrier_to_customer_days', 'delivery_time (days)', 'payment_value', 'seller_city', 'seller_state', 'product_category_name', 'product_height_cm','purchase_to_approval_days'])
y_approval = treino['approval_to_carrier_days']
y_carrier = treino['carrier_to_customer_days']

# Passo 4: Treinar modelos para 'approval_to_carrier_days' e 'carrier_to_customer_days'

# Modelo para 'approval_to_carrier_days'
model_approval = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=1500,         # Aumente o número de estimadores
    learning_rate=0.008,       # Reduza a taxa de aprendizado
    max_depth=9,              # Profundidade das árvores
    min_child_weight=9,      # Peso mínimo por folha
    subsample=0.6988466463923404,            # Subamostragem de linhas
    colsample_bytree=0.8533088440608811,     # Subamostragem de colunas
    reg_alpha=0.0009902391365606555,            # Regularização L1
    reg_lambda=0.7565432460670021,             # Regularização L2
    gamma=0.498568613419369                 # Reduzir a complexidade do modelo
)
model_approval.fit(X_intermediate, y_approval)

# Modelo para 'carrier_to_customer_days'
model_carrier = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=1600,         # Aumente o número de estimadores
    learning_rate=0.01,       # Reduza a taxa de aprendizado
    max_depth=7,              # Profundidade das árvores
    min_child_weight=10,      # Peso mínimo por folha
    subsample=0.6812398856987514,            # Subamostragem de linhas
    colsample_bytree=0.6432791404141186,     # Subamostragem de colunas
    reg_alpha=0.48691225203383853,            # Regularização L1
    reg_lambda=2.092554847706217,             # Regularização L2
    gamma=0.1956394716734175                 # Reduzir a complexidade do modelo
)
model_carrier.fit(X_intermediate, y_carrier)


# Passo 5: Preparar o conjunto de teste
teste = df

# Tratar valores ausentes nas variáveis numéricas
teste_numeric_cols = teste.select_dtypes(include=['float64', 'int64']).columns
teste[teste_numeric_cols] = teste[teste_numeric_cols].fillna(treino[numeric_cols].mean())  # Usando médias do treino

# Tratar valores ausentes nas variáveis categóricas
teste_categorical_cols = teste.select_dtypes(include=['object']).columns.drop('order_id', errors='ignore')
teste[teste_categorical_cols] = teste[teste_categorical_cols].fillna(treino[categorical_cols].mode().iloc[0])  # Usando modos do treino

# Remover 'order_id' das features
teste = teste.drop(columns=['order_id'], errors='ignore')

# Codificar variáveis categóricas no conjunto de teste
teste_categorical_cols = [col for col in categorical_cols if col in teste.columns]
teste[teste_categorical_cols] = ordinal_encoder.transform(teste[teste_categorical_cols])

# Garantir a mesma ordem de colunas
teste_intermediate = teste[X_intermediate.columns]

# Passo 6: Prever 'approval_to_carrier_days' e 'carrier_to_customer_days' no conjunto de teste
approval_pred = model_approval.predict(teste_intermediate)
carrier_pred = model_carrier.predict(teste_intermediate)

# Passo 7: Calcular o 'delivery_time (days)' como a soma das previsões e 'purchase_to_approval_days' do conjunto de teste
teste_final = teste_intermediate.copy()
teste_final['approval_to_carrier_days'] = approval_pred
teste_final['carrier_to_customer_days'] = carrier_pred
teste_final['purchase_to_approval_days'] = teste['purchase_to_approval_days']

# Calcular 'delivery_time (days)' somando as colunas
teste_final['delivery_time (days)'] = (
    teste_final['purchase_to_approval_days'] +
    teste_final['approval_to_carrier_days'] +
    teste_final['carrier_to_customer_days']
)

# Passo 8: Criar um DataFrame com 'order_id' e 'delivery_time (days)'
df_predictions = pd.DataFrame({
    'order_id': 'order_ids_test',
    'delivery_time (days)': teste_final['delivery_time (days)']
})

# Passo 9: Salvar as previsões em um arquivo CSV
df_predictions.to_csv('previsoes.csv', index=False)